# What is HAR ?
- Human activity recognition, or HAR for short, is a broad field of study concerned with identifying the specific movement or action of a person based on sensor data. 
- Movements are often typical activities performed indoors, such as walking, talking, standing, and sitting

# Why it is important ?

- Human activity recognition plays a significant role in human-to-human interaction and interpersonal relations. 
- Because it provides information about the identity of a person, their personality, and psychological state, it is difficult to extract. 
- The human ability to recognize another person’s activities is one of the main subjects of study of the scientific areas of computer vision and machine learning. As a result of this research, many applications, including video surveillance systems, human-computer interaction, and robotics for human behavior characterization, require a multiple activity recognition system.

#  Here are some example:

![](https://user-images.githubusercontent.com/11255376/71256359-d6ca1300-2373-11ea-812a-babb3b5b2ad5.gif)

![](https://learnopencv.com/wp-content/uploads/2021/07/yoga-poses.gif)

# What is VGG16?

- The VGG model, or VGGNet, that supports 16 layers is also referred to as VGG16, which is a convolutional neural network model proposed by A. Zisserman and K. Simonyan from the University of Oxford. These researchers published their model in the research paper titled, [“Very Deep Convolutional Networks for Large-Scale Image Recognition.”](https://arxiv.org/abs/1409.1556)

- The VGG16 model achieves almost 92.7% top-5 test accuracy in ImageNet. ImageNet is a dataset consisting of more than 14 million images belonging to nearly 1000 classes. Moreover, it was one of the most popular models submitted to ILSVRC-2014. It replaces the large kernel-sized filters with several 3×3 kernel-sized filters one after the other, thereby making significant improvements over AlexNet. The VGG16 model was trained using Nvidia Titan Black GPUs for multiple weeks.

- As mentioned above, the VGGNet-16 supports 16 layers and can classify images into 1000 object categories, including keyboard, animals, pencil, mouse, etc. Additionally, the model has an image input size of 224-by-224.  

# HAR using VGG16

### Importing library

In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

from tqdm import tqdm

from PIL import Image

from tensorflow.keras.utils import to_categorical

import seaborn as sns
import matplotlib.image as img
import matplotlib.pyplot as plt

In [1]:
pip list

Package                               Version                Editable project location
------------------------------------- ---------------------- -------------------------
absl-py                               1.0.0
accelerate                            0.9.0
access                                1.1.7
affine                                2.3.1
aiobotocore                           2.3.2
aiohttp                               3.8.1
aioitertools                          0.10.0
aiosignal                             1.2.0
albumentations                        1.1.0
alembic                               1.7.7
allennlp                              2.9.3
altair                                4.2.0
annoy                                 1.17.0
ansiwrap                              0.8.4
anyio                                 3.6.1
apache-beam                           2.38.0
aplus                                 0.11.0
appdirs                               1.4.4
argon2-cffi                   

# Setting up the path and loading csv files

In [ ]:
train_csv = pd.read_csv("../input/human-action-recognition-har-dataset/Human Action Recognition/Training_set.csv")
test_csv = pd.read_csv("../input/human-action-recognition-har-dataset/Human Action Recognition/Testing_set.csv")

In [ ]:
train_fol = glob.glob("../input/human-action-recognition-har-dataset/Human Action Recognition/train/*") 
test_fol = glob.glob("../input/human-action-recognition-har-dataset/Human Action Recognition/test/*")

In [ ]:
train_csv

In [ ]:
train_csv.label.value_counts()

In [ ]:
import plotly.express as px
l = train_csv.label.value_counts()
fig = px.pie(train_csv, values=l.values, names=l.index, title='Distribution of Human Activity')
fig.show()

In [ ]:
filename = train_csv['filename']

situation = train_csv['label']

In [ ]:
filename

In [ ]:
situation

# Creating a function to random take a image and display it with its label

In [ ]:
def disp():
    num = random.randint(1,10000)
    imgg = "Image_{}.jpg".format(num)
    train = "../input/human-action-recognition-har-dataset/Human Action Recognition/train/"
    if os.path.exists(train+imgg):
        testImage = img.imread(train+imgg)
        plt.imshow(testImage)
        plt.title("{}".format(train_csv.loc[train_csv['filename'] == "{}".format(imgg), 'label'].item()))

    else:
        #print(train+img)
        print("File Path not found \nSkipping the file!!")

In [ ]:
disp()

In [ ]:
disp()

In [ ]:
disp()

In [ ]:
disp()

In [ ]:
disp()

# Processing data

In [ ]:
img_data = []
img_label = []
length = len(train_fol)
for i in (range(len(train_fol)-1)):
    t = '../input/human-action-recognition-har-dataset/Human Action Recognition/train/' + filename[i]    
    temp_img = Image.open(t)
    img_data.append(np.asarray(temp_img.resize((160,160))))
    img_label.append(situation[i])

In [ ]:
inp_shape = (160, 160,3)

In [ ]:
iii = img_data
iii = np.asarray(iii)
type(iii)

In [ ]:
y_train = to_categorical(np.asarray(train_csv['label'].factorize()[0]))
print(y_train[0])

In [ ]:
# dir(tf.keras.applications)

In [ ]:
vgg_model = Sequential()

pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(160,160,3),
                   pooling='avg',classes=15,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

vgg_model.add(pretrained_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(512, activation='relu'))
vgg_model.add(Dense(15, activation='softmax'))

In [ ]:
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
vgg_model.summary()

In [ ]:
history = vgg_model.fit(iii,y_train, epochs=60)

In [ ]:
vgg_model.save_weights("model.h5")

In [ ]:
losss = history.history['loss']
plt.plot(losss)

In [ ]:
accu = history.history['accuracy']
plt.plot(accu)

# Custom Testing

In [ ]:
# Function to read images as array

def read_image(fn):
    image = Image.open(fn)
    return np.asarray(image.resize((160,160)))

In [ ]:
# Function to predict

def test_predict(test_image):
    result = vgg_model.predict(np.asarray([read_image(test_image)]))

    itemindex = np.where(result==np.max(result))
    prediction = itemindex[1][0]
    print("probability: "+str(np.max(result)*100) + "%\nPredicted class : ", prediction)

    image = img.imread(test_image)
    plt.imshow(image)
    plt.title(prediction)

In [ ]:
test_predict('../input/human-action-recognition-har-dataset/Human Action Recognition/test/Image_1010.jpg')

In [ ]:
test_predict('../input/human-action-recognition-har-dataset/Human Action Recognition/test/Image_101.jpg')

In [ ]:
test_predict('../input/human-action-recognition-har-dataset/Human Action Recognition/test/Image_1050.jpg')

![](https://c.tenor.com/MG_HgOuyCv8AAAAd/bye-goodbye.gif)

goodbye